In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *

# get key from file
my_key = get_key('koray_data')

# all usable env names
all_envs = ['data', 'staging', 'fourfront-webdev', 'fourfront-mastertest', 'fourfront-hotseat']
# get admin key from s3
#my_key = ff_utils.get_authentication_with_server({}, ff_env = 'data')

In [2]:
# Download link for old files are not working, checking all files to see if they are on S3 or not

q= '/search/?status=released&status=released+to+project&status=uploaded&type=FileMicroscopy&type=FileReference&type=FileCalibration'
all_fq = ff_utils.search_metadata(q, key=my_key)
print(len(all_fq))   


1445


In [3]:
from dcicutils import s3Utils
my_env = 'data'
my_s3_util = s3Utils(env=my_env)
raw_bucket = my_s3_util.raw_file_bucket
out_bucket = my_s3_util.outfile_bucket

counter = 0
my_bucket = raw_bucket
for i in reversed(all_fq): 
    counter += 1
    # check if file is in s3
    head_info = my_s3_util.does_key_exist(i['upload_key'], my_bucket)
    if not head_info:
        print(i['accession'], "does not have a file in S3")
        print()
    if counter % 100 == 0:
        print(counter)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [ ]:
# All files are on s3 
# the problem is with redirects

In [5]:
counter = 0
my_bucket = raw_bucket

#test cases all_fq = ['4DNFIJ5ILLUG', '4DNFIQMXJRKV']

problematic = []
pr_counter = 0
for i in reversed(all_fq): 
    counter += 1
    href = i['href']
    resp = ''
    try:
        e = ff_utils.get_metadata(href, key=my_key)
        resp = str(e)
    except Exception as ex:
        resp = str(ex)

    if 'elasticbeanstalk-fourfront-webprod-files' in resp:
        pass
    elif 'elasticbeanstalk-fourfront-webdev-files' in resp:
        problematic.append(i['uuid'])
        pr_counter += 1
    else:
        print(i['uuid'], 'other case', resp)
        print()
 
    if counter % 100 == 0:
        print(counter, pr_counter)

100 9
200 9
300 9
400 9
500 9
600 9
700 9
800 9
900 9


KeyboardInterrupt: 

In [6]:
print(len(problematic))
print(problematic)

9
['fe1181a4-2c20-48a9-8350-93ce8317f6f2', '37b870eb-0f63-4f0f-bb9b-34315f154160', '250b5413-fbea-4c33-9692-ad68bcfa771a', '66a4dfc2-05bf-4d38-99f8-23ebff0dd7c3', 'c48258bc-5949-485e-aeea-e6fa5cc30aa3', 'a5a40edb-8a36-4689-af9a-4d1aa1b0bd8d', 'cbc6e630-8677-4fdf-a70a-a33fb17c78ae', '4a6d10ee-2edb-4402-a98f-0edb1d58f5e9', '595763c6-58d3-4ec4-8f04-3dbb88ed4736']


In [7]:
import time
counter = 0
for pro in problematic:
    counter += 1
    status = ''
    status = [i for i in all_fq if i['uuid'] == pro][0]['status']
    ff_utils.patch_metadata({'status':'uploading'}, obj_id = pro, key = my_key)
    ff_utils.patch_metadata({'status':status}, obj_id = pro, key = my_key)
    if counter % 100 == 0:
        print(counter)
    